In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import pathlib 
dataframes = []
for f in pathlib.Path("/data/sjammi6/thesisproject/BlackHoleDiskProject/InitialData/routine2/data/processed_grids").glob("*.h5"):
    dataframes.append(str(f))
dataframes = [pd.read_hdf(f, key="df") for f in dataframes]
df_interpolated = pd.concat(dataframes, axis=0, ignore_index=True)
df_interpolated

In [ ]:
df_3d = pd.read_hdf("/data/sjammi6/thesisproject/BlackHoleDiskProject/InitialData/routine2/data/3D_data/all_data_routine1.h5", key="df")
df_3d

In [ ]:
column_names = [
    "x",
    "z",
    "r",
    "theta",
    "alpha",
    "psi",
    "q",
    "betak",
    "betat",
    "dbetatdr",
    "dbetatdt",
    "He",
    "Hf",
    "Omega",
    "b2",
    "rho",
]
df_2d = pd.read_csv("/data/sjammi6/thesisproject/BlackHoleDiskProject/InitialData/October_15_2024_22-05/2D_data/all_vars.dat", delimiter=r"\s+", header=None, names=column_names)
df_2d

In [ ]:
df_2d["phi"] = np.float64(0.0)
df_2d["y"] = np.float64(0.0)
df_2d["gxx"] = (df_2d.psi**4 * np.exp(2 * df_2d.q)) * (
    ((df_2d.x / df_2d.r) ** 2) + ((df_2d.z * np.cos(df_2d.phi) / df_2d.r) ** 2)
) + np.sin(df_2d.phi) ** 2 * df_2d.psi**4
df_2d["gxy"] = (
    df_2d.psi**4
    * np.exp(2 * df_2d.q)
    * (df_2d.x * df_2d.y / df_2d.r**2 + df_2d.z**2 * np.cos(df_2d.phi) * np.sin(df_2d.phi) / df_2d.r**2)
    - np.sin(df_2d.phi) * np.cos(df_2d.phi) * df_2d.psi**4
)
df_2d["gxz"] = (
    df_2d.psi**4
    * np.exp(2 * df_2d.q)
    * (
        df_2d.x * df_2d.z / df_2d.r**2
        - df_2d.z * np.cos(df_2d.phi) * np.sqrt(df_2d.x**2 + df_2d.y**2) / df_2d.r**2
    )
)
df_2d["gyy"] = (
    df_2d.psi**4
    * np.exp(2 * df_2d.q)
    * (df_2d.y**2 / df_2d.r**2 + df_2d.z**2 * np.sin(df_2d.phi) ** 2 / df_2d.r**2)
    + np.cos(df_2d.phi) ** 2 * df_2d.psi**4
)
df_2d["gyz"] = (
    df_2d.psi**4
    * np.exp(2 * df_2d.q)
    * (
        df_2d.y * df_2d.z / df_2d.r**2
        - df_2d.z * np.sin(df_2d.phi) * np.sqrt(df_2d.x**2 + df_2d.y**2) / df_2d.r**2
    )
)
df_2d["gzz"] = (
    df_2d.psi**4 * np.exp(2 * df_2d.q) * (df_2d.x**2 + df_2d.y**2) / df_2d.r**2
    + df_2d.psi**4 * np.exp(2 * df_2d.q) * df_2d.z**2 / df_2d.r**2
)
df_2d_new_xplane = df_2d[((df_2d.theta == dict(df_2d.theta.describe())["max"])) & (df_2d.phi == np.float64(0))]
df_2d_new_xplane

In [ ]:
df_3d_xplane = df_3d[(df_3d.theta == dict(df_3d.theta.describe())["50%"]) & (df_3d.phi == dict(df_3d.phi.describe())["min"])]
df_3d_xplane = df_3d_xplane.sort_values(by="r").drop_duplicates()
df_3d_xplane.sort_values(by="r")
df_3d_xplane

In [ ]:
df_interpolated_xplane = df_interpolated[(df_interpolated.theta == dict(df_interpolated.theta.describe())["50%"]) & (df_interpolated.phi == dict(df_interpolated.phi.describe())["min"])]
df_interpolated_xplane = df_interpolated_xplane.sort_values(by="r").drop_duplicates()
df_interpolated_xplane.sort_values(by="r")
df_interpolated_xplane

In [4]:
df_interpolated["r"] = np.sqrt(df_interpolated.x**2 + df_interpolated.y**2 + df_interpolated.z**2)
df_interpolated["theta"] = np.where(df_interpolated.r != np.float64(0), np.arccos(df_interpolated.z / df_interpolated.r), np.float64(0))
df_interpolated["phi"] = np.arctan2(df_interpolated.y, df_interpolated.x) % (2 * np.pi)

df_interpolated = df_interpolated.sort_values(['r', 'theta', 'phi'])

In [5]:
df_interpolated = df_interpolated.map(lambda x: np.float64(0) if abs(x) <= 1e-10 else x)


In [ ]:
interpolatedstats = dict(df_interpolated.describe())
interpolatedstats

In [ ]:
list(df_3d.columns)[3:]

In [ ]:
for col in list(df_3d.columns)[3:]:
    fig, ax = plt.subplots(figsize=(20,12))
    plt.subplots_adjust(right=0.75)
    plt.xlim(1e-6, 60)
    plt.xscale('log')
    #ax.plot(df_interpolated_xplane["x"], df_interpolated_xplane["rho"], color='#644117', label="interpolated data")
    #ax.plot(df_2d_new_xplane["r"], df_2d_new_xplane[col], color='b', label="2d data")
    ax.plot(df_3d_xplane["r"], df_3d_xplane[col], color='g', linestyle="--", label="3d data")
    ax.plot(df_interpolated_xplane["x"], df_interpolated_xplane[col], color='#644117', linestyle=":", label="interpolated data")
    ax.axvline(x=np.float64(12.0), color="#bc8f8f", linestyle="--", label="Inner Radius of Disk")
    ax.axvline(x=np.float64(30.0), color="#bc8f8f", linestyle="--", label="Inner Radius of Disk")
    ax.axvline(x=np.float64(0.3), color="#bb6528", linestyle="--", label="Horizon")
    plt.xlabel("r")
    plt.ylabel(col)
    plt.title("{} vs r (y = 0, z= 0)".format(col))
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

In [178]:
def locate_point(
    r_arr, theta_arr, phi_arr, p
):  # r_arr, theta_arr, phi_arr define the ID grid, p defines a point in the new grid
    # p should be a tuple (x,y,z) in cartesian coords
    phi_arr = np.append(phi_arr[-2], np.append(phi_arr, phi_arr[1]))
    x, y, z = p
    r_0 = np.sqrt(x**2 + y**2 + z**2)
    theta_0 = np.arccos(z / r_0)
    skip = False
    origin = False
    phi_0 = np.arctan2(y, x)
    phi_0 = phi_0 % (2 * np.pi)  # ensures phi takes values between [0,2pi]
    ts = np.zeros(2, dtype=np.float64)
    ps = np.zeros(2, dtype=np.float64)
    rs = np.zeros(2, dtype=np.float64)
    for i in range(1, len(phi_arr) - 1):
        if phi_0 == phi_arr[i]:
            ps[0] = phi_arr[i - 1]
            ps[1] = phi_arr[i + 1]
            break
        if phi_arr[i] < phi_0 < phi_arr[i + 1]:
            ps[0] = phi_arr[i]
            ps[1] = phi_arr[i + 1]
            break
    if theta_0 == theta_arr[0]:
        ts[0] = theta_arr[1]
        ps = np.append(ps, np.array([phi_arr[-2] - ps[0], phi_arr[-2] - ps[1]]))
        skip = True
    if theta_0 == theta_arr[-1]:
        ts[0] = theta_arr[-2]
        ps = np.append(ps, np.array([phi_arr[-2] - ps[0], phi_arr[-2] - ps[1]]))
        skip = True

    if not skip:
        for i in range(len(theta_arr)):
            if theta_0 == theta_arr[i]:
                ts[0] = theta_arr[i - 1]
                ts[1] = theta_arr[i + 1] 
                break
            if theta_arr[i] < theta_0 < theta_arr[i + 1]:
                ts[0] = theta_arr[i]
                ts[1] = theta_arr[i + 1]
                break

    for i in range(len(r_arr)):
        if r_0 == r_arr[i]:
            rs[0] = r_arr[i - 1]
            rs[1] = r_arr[i + 1]
            break
        if r_arr[i] < r_0 < r_arr[i + 1]:
            rs[0] = r_arr[i]
            rs[1] = r_arr[i + 1]
            break
    tc = 8
    if rs[0] == np.float64(0.0):
        origin = True
        tc = 5

    combinations = np.zeros((tc, 3), dtype=np.float64)

    if origin:
        combinations[0] = np.array([np.float64(0), np.float64(0), np.float64(0)])
        if skip:
            combinations[1] = np.array([rs[1], ts[0], ps[0]])
            combinations[2] = np.array([rs[1], ts[0], ps[1]])
            combinations[3] = np.array([rs[1], ts[0], ps[2]])
            combinations[4] = np.array([rs[1], ts[0], ps[3]])
        else:
            combinations[1] = np.array([rs[1], ts[0], ps[0]])
            combinations[2] = np.array([rs[1], ts[0], ps[1]])
            combinations[3] = np.array([rs[1], ts[1], ps[0]])
            combinations[4] = np.array([rs[1], ts[1], ps[1]])
    else:
        if skip:
            combinations[0] = np.array([rs[0], ts[0], ps[0]])
            combinations[1] = np.array([rs[0], ts[0], ps[1]])
            combinations[2] = np.array([rs[0], ts[0], ps[2]])
            combinations[3] = np.array([rs[0], ts[0], ps[3]])
            combinations[4] = np.array([rs[1], ts[0], ps[0]])
            combinations[5] = np.array([rs[1], ts[0], ps[1]])
            combinations[6] = np.array([rs[1], ts[0], ps[2]])
            combinations[7] = np.array([rs[1], ts[0], ps[3]])
        else:
            combinations[0] = np.array([rs[0], ts[0], ps[0]])
            combinations[1] = np.array([rs[0], ts[0], ps[1]])
            combinations[2] = np.array([rs[0], ts[1], ps[0]])
            combinations[3] = np.array([rs[0], ts[1], ps[1]])
            combinations[4] = np.array([rs[1], ts[0], ps[0]])
            combinations[5] = np.array([rs[1], ts[0], ps[1]])
            combinations[6] = np.array([rs[1], ts[1], ps[0]])
            combinations[7] = np.array([rs[1], ts[1], ps[1]])

    return combinations 




In [43]:
def calc_weighted_average(points, scalar_vals, p):
    d = np.sum(np.abs((points - p)), axis=1)
    if any(d <= 1e-9):
        return scalar_vals[np.argmin(d)]
    weights = 1 / d
    weights = weights.reshape((-1, 1))
    return np.sum(weights * scalar_vals, axis=0) / np.sum(weights)

In [15]:
x = np.linspace(0, 10, 11)
y = np.linspace(0, 10, 11)
z = np.linspace(0, 10, 11)

In [16]:
cartesian_points = np.array([[i, j, k] for i in x for j in y for k in z])

In [ ]:
cartesian_points

In [46]:
def cart_to_spher(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r) if r != 0 else np.float64(0)
    phi = np.arctan2(y, x) % (2 * np.pi)
    return np.array([r, theta, phi])

In [ ]:
spherical_points = np.array([cart_to_spher(i, j, k) for i, j, k in cartesian_points])

In [20]:
spherical_points[0] = np.array([np.float64(0), np.float64(0), np.float64(0)])

In [ ]:
spherical_points

In [22]:
p = np.array([3.2, 2.9, 3.1])

In [23]:
r_arr = np.unique(spherical_points[:, 0])
phi_arr = np.unique(spherical_points[:, 2])
theta_arr = np.unique(spherical_points[:, 1])

In [24]:
pdata = locate_point(r_arr, theta_arr, phi_arr, p)

In [ ]:
pdata

In [25]:
def spherical2cart(r, t, p):
    x = r * np.sin(t) * np.cos(p)
    y = r * np.sin(t) * np.sin(p)
    z = r * np.cos(t)
    return (x, y, z)


In [26]:
carpdata = np.array([spherical2cart(i, j, k) for i, j, k in pdata])

In [ ]:
carpdata

In [47]:
cardatatwo = np.array([spherical2cart(i, j, k) for i, j, k in spherical_points])

In [ ]:
cardatatwo

In [ ]:
pdata

In [14]:
def locate_point(
    r_arr, theta_arr, phi_arr, p
):  # r_arr, theta_arr, phi_arr define the ID grid, p defines a point in the new grid
    # p should be a tuple (x,y,z) in cartesian coords
    phi_arr = np.append(phi_arr[-2], np.append(phi_arr, phi_arr[1]))
    x, y, z = p
    r_0 = np.sqrt(x**2 + y**2 + z**2)
    theta_0 = np.arccos(z / r_0)
    skip = False
    origin = False
    phi_0 = np.arctan2(y, x)
    phi_0 = phi_0 % (2 * np.pi)  # ensures phi takes values between [0,2pi]
    ts = []
    ps = []
    rs = []
    for i in range(1, len(phi_arr) - 1):
        if phi_0 == phi_arr[i]:
            ps.append(phi_arr[i - 1])
            ps.append(phi_arr[i + 1])
            break
        if phi_arr[i] < phi_0 < phi_arr[i + 1]:
            ps.append(phi_arr[i])
            ps.append(phi_arr[i + 1])
            break
    ps = np.array(ps)
    if theta_0 == theta_arr[0]:
        ts.append(theta_arr[1])
        ps = np.append(ps, np.array([phi_arr[-2] - ps[0], phi_arr[-2] - ps[1]]))
        skip = True
    if theta_0 == theta_arr[-1]:
        ts.append(theta_arr[-2])
        ps = np.append(ps, np.array([phi_arr[-2] - ps[0], phi_arr[-2] - ps[1]]))
        skip = True
    if not skip:
        for i in range(len(theta_arr)):
            if theta_0 == theta_arr[i]:
                ts.append(theta_arr[i - 1])
                ts.append(theta_arr[i + 1])
                #ts[0] = theta_arr[i - 1]
                #ts[1] = theta_arr[i + 1] 
                break
            if theta_arr[i] < theta_0 < theta_arr[i + 1]:
                ts.append(theta_arr[i])
                ts.append(theta_arr[i + 1])
                #ts[0] = theta_arr[i]
                #ts[1] = theta_arr[i + 1]
                break
    ts = np.array(ts)
    for i in range(len(r_arr)):
        if r_0 == r_arr[i]:
            rs.append(r_arr[i - 1])
            rs.append(r_arr[i + 1])
            #rs[0] = r_arr[i - 1]
            #rs[1] = r_arr[i + 1]
            break
        if r_arr[i] < r_0 < r_arr[i + 1]:
            rs.append(r_arr[i])
            rs.append(r_arr[i + 1])
            #rs[0] = r_arr[i]
            #rs[1] = r_arr[i + 1]
            break
    rs = np.array(rs)
    tc = 8
    combinations = []

    if origin:
        combinations.append(np.array([np.float64(0), np.float64(0), np.float64(0)]))
        for t in ts:
            for p in ps:
                combinations.append(np.array([rs[1], t, p]))
    else:
        for r in rs:
            for t in ts:
                for p in ps:
                    combinations.append(np.array([r, t, p]))

    return np.array(combinations)

In [31]:
cartes_spher_cart = np.array([spherical2cart(i, j, k) for i, j, k in spherical_points])

In [ ]:
cartes_spher_cart

In [ ]:
r_arr

In [177]:
import numpy as np

def locate_point(r_arr, theta_arr, phi_arr, p):
    x, y, z = p
    r_0 = np.sqrt(x**2 + y**2 + z**2)
    theta_0 = np.arccos(z / r_0) if r_0 != 0 else np.float64(0)
    phi_0 = np.arctan2(y, x) % (2 * np.pi)

    # Find nearest indices
    r_idx_1 = np.searchsorted(r_arr, r_0) - 1
    r_idx_2 = np.searchsorted(r_arr, r_0, side="right")
    theta_idx_1 = np.searchsorted(theta_arr, theta_0) - 1
    theta_idx_2 = np.searchsorted(theta_arr, theta_0, side="right")
    phi_idx_1 = np.searchsorted(phi_arr, phi_0) - 1
    phi_idx_2 = np.searchsorted(phi_arr, phi_0, side="right")
    
    theta_edge_case = False
    if theta_idx_1 == -1:
        theta_idx_1 = 1
        theta_edge_case = True
    if theta_idx_2 == len(theta_arr) + 1:
        theta_idx_2 = theta_idx_1
        theta_edge_case = True
    if phi_idx_1 == -1:
        phi_idx_1 = len(phi_arr) - 2
    if phi_idx_2 == len(phi_arr) + 1:
        phi_idx_2 = 1
    # Handle edge cases
    #theta_idx = min(max(theta_idx, 1), len(theta_arr) - 1)
    #phi_idx = min(max(phi_idx, 1), len(phi_arr) - 1)

    # Get surrounding points
    rs = [r_arr[r_idx_1], r_arr[r_idx_2]]
    print(rs)
    ts = [theta_arr[theta_idx_1], theta_arr[theta_idx_2]]
    print(ts)
    #ts = theta_arr[theta_idx-1:theta_idx+1]
    ps = [phi_arr[phi_idx_1], phi_arr[phi_idx_2]]
    if theta_edge_case:
        ps.append(phi_arr[-2] - ps[0]) 
        ps.append(phi_arr[-2] - ps[1])
    #ps = phi_arr[phi_idx-1:phi_idx+1]
    print(ps)
    # Generate all combinations
    combinations = np.array(np.meshgrid(rs, ts, ps)).T.reshape(-1, 3)
    z = combinations[:, 0] == np.float64(0)
    combinations[z] = np.array([np.float64(0), np.float64(0), np.float64(0)])
    combinations = np.unique(combinations, axis=0)
    return combinations


In [158]:
r_arr = np.unique(spherical_points[:, 0])
theta_arr = np.unique(spherical_points[:, 1])
phi_arr = np.unique(spherical_points[:, 2])

In [159]:
p = np.array([3.2, 2.9, 3.1])

In [160]:
xs = [np.float64(3), np.float64(4)]
ys = [np.float64(2), np.float64(3)]
zs = [np.float64(3), np.float64(4)]
tmpcombs = np.array(np.meshgrid(xs, ys, zs)).T.reshape(-1, 3)

In [ ]:
tmpcombs

In [162]:
tmpcombs_spherical = np.array([cart_to_spher(i, j, k) for i, j, k in tmpcombs])

In [ ]:
tmpcombs_spherical

In [ ]:
res = locate_point(r_arr, theta_arr, phi_arr, p)

In [ ]:
res

In [57]:
cart_res = np.array([spherical2cart(i, j, k) for i, j, k in res])

In [ ]:
cart_res

In [ ]:
cart_to_spher(p[0], p[1], p[2])

In [173]:
d = res[:, 0] == tmp
res[d] = np.array([np.float64(0), np.float64(0), np.float64(0)])

In [ ]:
tmp = res[0][0]
tmp

In [ ]:
res

In [ ]:
np.unique(res, axis=0)

In [63]:
tmpa = np.linspace(0, 10, 11)

In [ ]:
tmpa

In [149]:
ti = np.searchsorted(tmpa, 10) - 1
tt = np.searchsorted(tmpa, 10, side='right')

In [ ]:
tt

In [ ]:
ti

In [76]:
idx = min(max(ti, 1), len(tmpa) - 1)
#    r_idx = min(max(r_idx, 1), len(r_arr) - 1)
#    theta_idx = min(max(theta_idx, 1), len(theta_arr) - 1)
#    phi_idx = min(max(phi_idx, 1), len(phi_arr) - 1)


In [ ]:
idx

In [ ]:
list(tmpa)[-1]

In [110]:
a= [i for i in range(10)]

In [ ]:
a[-1]

In [113]:
a = [1, 1]
b = [2, 3]
c = [4, 5]
d = [6, 7]


In [123]:
tmpcombs = np.array(np.meshgrid(a, c, d)).T.reshape(-1, 3)

In [ ]:
tmpcombs

In [ ]:
np.unique(tmpcombs, axis=0)

In [ ]:
foo = []
for y in c:
    for z in d:
        for x in b:
            foo.append([x, y, z])
            print(x, y, z)

In [ ]:
np.sort(foo)

In [ ]:
foo

In [16]:
p = [np.float64(13.2), np.float64(0), np.float64(0)]

In [24]:
def locate_point(r_arr, theta_arr, phi_arr, p):
    x, y, z = p
    r_0 = np.sqrt(x**2 + y**2 + z**2)
    theta_0 = np.arccos(z / r_0) if r_0 != 0 else np.float64(0)
    phi_0 = np.arctan2(y, x) % (2 * np.pi)

    # Find nearest indices
    r_idx_1 = np.searchsorted(r_arr, r_0) - 1
    r_idx_2 = np.searchsorted(r_arr, r_0, side="right")
    theta_idx_1 = np.searchsorted(theta_arr, theta_0) - 1
    theta_idx_2 = np.searchsorted(theta_arr, theta_0, side="right")
    phi_idx_1 = np.searchsorted(phi_arr, phi_0) - 1
    phi_idx_2 = np.searchsorted(phi_arr, phi_0, side="right")
    
    theta_edge_case = False
    if theta_idx_1 == -1:
        theta_idx_1 = 1
        theta_edge_case = True
    if theta_idx_2 == len(theta_arr) + 1:
        theta_idx_2 = theta_idx_1
        theta_edge_case = True
    if phi_idx_1 == -1:
        phi_idx_1 = len(phi_arr) - 2
    if phi_idx_2 == len(phi_arr) + 1:
        phi_idx_2 = 1
    # Handle edge cases

    # Get surrounding points
    rs = [r_arr[r_idx_1], r_arr[r_idx_2]]
    ts = [theta_arr[theta_idx_1], theta_arr[theta_idx_2]]
    ps = [phi_arr[phi_idx_1], phi_arr[phi_idx_2]]
    if theta_edge_case:
        ps.append(phi_arr[-2] - ps[0]) 
        ps.append(phi_arr[-2] - ps[1])
    # Generate all combinations
    combinations = np.array(np.meshgrid(rs, ts, ps)).T.reshape(-1, 3)
    z = combinations[:, 0] == np.float64(0)
    combinations[z] = np.array([np.float64(0), np.float64(0), np.float64(0)])
    combinations = np.unique(combinations, axis=0)
    return combinations


def calc_weighted_average(points, scalar_vals, p):
    d = np.sum(np.abs((points - p)), axis=1)
    if np.any(d <= 1e-6):
        return scalar_vals[np.argmin(d)]
    weights = 1 / d
    weights = weights.reshape((-1, 1))
    return np.sum(weights * scalar_vals, axis=0) / np.sum(weights)


def spherical2cart(r, t, p):
    x = r * np.sin(t) * np.cos(p)
    y = r * np.sin(t) * np.sin(p)
    z = r * np.cos(t)
    return (x, y, z)


def interpolate_point(p, df, rad_arr, theta_arr, phi_arr, idx_point_map):
    surroundingpoints = locate_point(rad_arr, theta_arr, phi_arr, p)  # numpy array
    carpointdata = []
    scalardata = []
    for point in surroundingpoints:
        tmpr, tmpt, tmpp = point
        rowIndex = idx_point_map[(tmpr, tmpt, tmpp)]
        rowdata = df.iloc[rowIndex]
        tmpx, tmpy, tmpz = spherical2cart(tmpr, tmpt, tmpp)
        carpointdata.append([tmpx, tmpy, tmpz])
        scalardata.append(rowdata[3:])
    carpointdata = np.array(carpointdata)
    scalardata = np.array(scalardata)
    interpolateddata = calc_weighted_average(carpointdata, scalardata, p)
    return np.hstack((p, interpolateddata))


In [ ]:
print("PREPROCESSING DATA")
rad_arr = np.sort(df_3d.r.unique())
theta_arr = np.sort(df_3d.theta.unique())
phi_arr = np.sort(df_3d.phi.unique())
idx_point_map = {
    (r, theta, phi): index
    for index, (r, theta, phi) in enumerate(zip(df_3d["r"], df_3d["theta"], df_3d["phi"]))
}

In [ ]:
locate_point(rad_arr, theta_arr, phi_arr, [13.2, 0, 0])

In [ ]:
interpolate_point([13.2, 0, 0], df_3d, rad_arr, theta_arr, phi_arr, idx_point_map)

In [53]:
surroundingpoints = locate_point(rad_arr, theta_arr, phi_arr, p)  # numpy array
pnew = cart_to_spher(p[0], p[1], p[2])
#carpointdata = []
scalardata = []
for point in surroundingpoints:
    tmpr, tmpt, tmpp = point
    rowIndex = idx_point_map[(tmpr, tmpt, tmpp)]
    rowdata = df_3d.iloc[rowIndex]
    #tmpx, tmpy, tmpz = spherical2cart(tmpr, tmpt, tmpp)
    #carpointdata.append([tmpx, tmpy, tmpz])
    scalardata.append(rowdata[3:])
scalardata_two = np.array(scalardata)
interpolateddata = calc_weighted_average(surroundingpoints, scalardata_two, pnew)

In [ ]:
df_3d_xplane[(df_3d.r < np.float64(15)) & (df_3d.r > np.float64(10)) & (df_3d.theta == np.float64(0))]

In [ ]:
surroundingpoints

In [ ]:
df_3d.phi.unique()

In [ ]:
6.346652e-02

In [ ]:
df_3d.iloc[6322401]

In [ ]:
scalardata

In [35]:
cols = list(df_3d.columns[3:])

In [ ]:
cols

In [ ]:
df_3d_xplane = df_3d[(df_3d.theta == dict(df_3d.theta.describe())["50%"]) & (df_3d.r == np.float64(13.078883))]
df_3d_xplane = df_3d_xplane.sort_values(by="phi").drop_duplicates()
df_3d_xplane.sort_values(by="phi")
df_3d_xplane